# Demo for PyTorch Mask RCNN
Base Repo:  https://github.com/multimodallearning/pytorch-mask-rcnn

Sample Repo:  https://github.com/michhar/pytorch-mask-rcnn-samples

**Note:  Run this notebook inside of the Sample Repo after cloning**

## Prerequisites

1.  NVIDIA CUDA Toolkit setup ([NVIDIA CUDA Toolkit Docs](https://docs.nvidia.com/cuda/index.html))
2.  PyTorch (tested with version below) - **note**:  further testing required as PyTorch has updated to 1.0 which is in preview (as of this notebook update) and the code was originally using 0.4.  It's likely the model class and training script will need to be updated as well.  In addition, this notebook is testing a custom build of PyTorch for the machine.  Troubleshooting is happening for the custom extension builds as well. (2018-09-28)

## Installs and Setup

In [ ]:
# # If not built for the system already
# ! pip install torch torchvision

In [ ]:
import sys
sys.prefix

In [16]:
import torch
torch.__version__

'1.0.0a0+c35f85a'

**Build two extensions**

* nms
* roi align

In [21]:
! source ~/.bash_profile

In [ ]:
! export PATH=/Developer/NVIDIA/CUDA-10.0/bin:$PATH &&\
    export LD_LIBRARY_PATH=//Developer/NVIDIA/CUDA-10.0/lib:$LD_LIBRARY_PATH &&\
    export CUDA_BIN_PATH=/Developer/NVIDIA/CUDA-10.0/bin &&\
    export CUDA_TOOLKIT_ROOT_DIR=/Developer/NVIDIA/CUDA-10.0 &&\
    export CUDNN_LIB_DIR=//Developer/NVIDIA/CUDA-10.0/lib &&\
    export USE_CUDA=1 &&\
    export CC=clang &&\
    export CXX=clang++ &&\
    cd nms/src/cuda/ && nvcc -c -o nms_kernel.cu.o nms_kernel.cu -x cu -Xcompiler -fPIC &&\
    cd ../../ &&\
    CC=clang CXX=clang++ python3 build.py

For the next cuda compiler command, `-arch` flag refers to architectures listed at https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#gpu-compilation.

In [ ]:
! export PATH=/Developer/NVIDIA/CUDA-10.0/bin:$PATH &&\
    export LD_LIBRARY_PATH=//Developer/NVIDIA/CUDA-10.0/lib:$LD_LIBRARY_PATH &&\
    export CUDA_BIN_PATH=/Developer/NVIDIA/CUDA-10.0/bin &&\
    export CUDA_TOOLKIT_ROOT_DIR=/Developer/NVIDIA/CUDA-10.0 &&\
    export CUDNN_LIB_DIR=//Developer/NVIDIA/CUDA-10.0/lib &&\
    export USE_CUDA=1 &&\
    export CC=clang &&\
    export CXX=clang++ &&\
    cd roialign/roi_align/src/cuda/ &&\
    nvcc -c -o crop_and_resize_kernel.cu.o crop_and_resize_kernel.cu -x cu -Xcompiler -fPIC &&\
    cd ../../ &&\
    python3 build.py

In [ ]:
! ls

In [ ]:
! cd roialign/roi_align/_ext/crop_and_resize/ && ls

In [ ]:
# ! wget http://images.cocodataset.org/zips/train2014.zip

In [ ]:
## If 
# ! rm -fr cocoapi
# ! rm -fr pycocotools
# ! export CC="/usr/local/opt/llvm/bin/clang" &&\
#     git clone https://github.com/cocodataset/cocoapi.git &&\
#     cd cocoapi/PythonAPI &&\
#     ln -s cocoapi/PythonAPI/pycocotools/ pycocotools &&\
#     python3 setup.py build_ext --inplace

In [ ]:
! rm -fr coco
! rm -fr pycocotools
! export CC="/usr/local/opt/llvm/bin/clang" &&\
    git clone https://github.com/waleedka/coco.git &&\
    cd coco/PythonAPI &&\
    python3 setup.py build_ext --inplace

In [ ]:
! rm -fr pycocotools
! ln -s coco/PythonAPI/pycocotools/ pycocotools

### Download the Model

Here:  https://drive.google.com/open?id=1VV6WgX_RNl6a9Yi9-Pe7ZyVKHRJZSKkm

And upload to the root directory of the repo on this machine (where this notebook lives).

## Setup and Run Inference

In [ ]:
# Am I running 32 or 64 bit Python
import struct
print(struct.calcsize("P") * 8)

In [25]:
# %load_ext autoreload
# %autoreload
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import glob
from io import BytesIO
import requests
from PIL import Image
%matplotlib inline

import coco
# from pycocotools import coco
import utils
import model as modellib
import visualize

import torch
import pycocotools

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to trained weights file
# Download this file and place in the root of your
# project (See README file for details)
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "models", "mask_rcnn_coco.pth")

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    # GPU_COUNT = 0 for CPU
    GPU_COUNT = 0
    IMAGES_PER_GPU = 1
    COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.pth")


config = InferenceConfig()
config.display()

# Create model object.
model = modellib.MaskRCNN(model_dir=MODEL_DIR, config=config)
model = model.to(device=device)

cpu

Configurations:
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COCO_MODEL_PATH                /Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/mask_rcnn_coco.pth
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      0
IMAGENET_MODEL_PATH            /Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/resnet50_imagenet.pth
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_MIN_DIM                  800
IMAGE_PADDING                  True
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL

In [26]:
# Load weights trained on MS-COCO
print(COCO_MODEL_PATH)
model.load_state_dict(torch.load(COCO_MODEL_PATH))

/Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/models/mask_rcnn_coco.pth


In [27]:
# COCO Class names
# Index of the class in the list is its ID. For example, to get ID of
# the teddy bear class, use: class_names.index('teddy bear')
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

# Load a random image from the images folder
# file_names = glob.glob(os.path.join('images', '*.jpg'))
# image = skimage.io.imread(os.path.join(random.choice(file_names)))

# Or load file from the internet
req = requests.get('https://cdn.pixabay.com/photo/2015/06/20/13/55/man-815795__340.jpg')
image = np.asarray(Image.open(BytesIO(req.content)))

In [28]:
# Run detection
results = model.detect([image])

FatalError: $ Torch: invalid memory size -- maybe an overflow? at /Users/micheleenharris/Documents/bin/github/pytorch-mask-rcnn-samples/pytorch/aten/src/TH/THGeneral.cpp:191

In [ ]:
# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                            class_names, r['scores'])
plt.show()

## Run an actual training experiment

In [ ]:
# ! python coco.py train --dataset=mask_rcnn_coco.pth --model=imagenet